In [1]:
4000+4

4004

In [3]:
!pip install pydicom matplotlib numpy pillow


   ---------------------------------------- 0.0/2.4 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.4 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.4 MB ? eta -:--:--
    --------------------------------------- 0.0/2.4 MB 262.6 kB/s eta 0:00:09
   - -------------------------------------- 0.1/2.4 MB 409.6 kB/s eta 0:00:06
   -- ------------------------------------- 0.1/2.4 MB 654.9 kB/s eta 0:00:04
   ---- ----------------------------------- 0.3/2.4 MB 1.1 MB/s eta 0:00:03
   -------- ------------------------------- 0.5/2.4 MB 1.9 MB/s eta 0:00:02
   --------------- ------------------------ 0.9/2.4 MB 2.8 MB/s eta 0:00:01
   ----------------------- ---------------- 1.4/2.4 MB 3.8 MB/s eta 0:00:01
   ----------------------------- ---------- 1.8/2.4 MB 4.3 MB/s eta 0:00:01
   ------------------------------ --------- 1.8/2.4 MB 3.9 MB/s eta 0:00:01
   ---------------------------------- ----- 2.1/2.4 MB 4.0 MB/s eta 0:00:01
   ------------------------

In [6]:
import os
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
from PIL import Image

# Directory where images are stored
input_directory = r'F:\MLpractrice\ct\data'
output_directory = r'F:\MLpractrice\ct\output'  # Output directory for color-mapped images

# Create the output directory if it doesn't exist
if not os.path.exists(output_directory):
    os.makedirs(output_directory)

# List all image files (jpg and png) in the directory
image_files = [f for f in os.listdir(input_directory) if f.endswith('.jpg') or f.endswith('.png')]

# Debugging: Check if image files are found
if not image_files:
    print("No image files found in the specified directory.")
else:
    print(f"Found {len(image_files)} image files.")

# Process each image file
for image_file in image_files:
    try:
        image_path = os.path.join(input_directory, image_file)
        
        # Load the image using PIL
        image = Image.open(image_path)
        
        # Convert image to grayscale (if it's not already in grayscale)
        ct_image = np.array(image.convert('L'))  # Convert to grayscale (L mode)
        
        # Normalize the image (Optional: normalize to the range 0-1 for better color mapping)
        ct_image_normalized = (ct_image - np.min(ct_image)) / (np.max(ct_image) - np.min(ct_image))
        
        # Apply a color map (e.g., 'jet', 'hot', etc.)
        colored_image = cm.jet(ct_image_normalized)  # You can change 'jet' to another color map
        
        # Convert the color-mapped image from (Height, Width, Channels) to (Height, Width)
        colored_image_rgb = (colored_image[:, :, :3] * 255).astype(np.uint8)  # Remove alpha channel
        
        # Save the color-mapped image as PNG
        output_image_path = os.path.join(output_directory, image_file.replace('.jpg', '_colored.png').replace('.png', '_colored.png'))
        Image.fromarray(colored_image_rgb).save(output_image_path)

        print(f"Processed and saved {image_file} as {output_image_path}")
    
    except Exception as e:
        print(f"Error processing {image_file}: {e}")

print("Processing complete.")


Found 2 image files.
Processed and saved 2.png as F:\MLpractrice\ct\output\2_colored.png
Processed and saved istockphoto-502727050-612x612.jpg as F:\MLpractrice\ct\output\istockphoto-502727050-612x612_colored_colored.png
Processing complete.


In [7]:
import os
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
from PIL import Image
import cv2
from skimage.feature import greycomatrix, greycoprops
import SimpleITK as sitk

# Define input and output directories
input_directory = r'F:\MLpractrice\ct\data'
output_directory = r'F:\MLpractrice\ct\output'

# Create the output directory if it doesn't exist
if not os.path.exists(output_directory):
    os.makedirs(output_directory)

# List all image files (jpg and png) in the directory
image_files = [f for f in os.listdir(input_directory) if f.endswith('.jpg') or f.endswith('.png')]

# Debugging: Check if image files are found
if not image_files:
    print("No image files found in the specified directory.")
else:
    print(f"Found {len(image_files)} image files.")

# Method 1: Basic Color Map Application
def apply_color_map(ct_image, cmap='jet'):
    ct_image_normalized = (ct_image - np.min(ct_image)) / (np.max(ct_image) - np.min(ct_image))
    colored_image = cm.get_cmap(cmap)(ct_image_normalized)
    return (colored_image[:, :, :3] * 255).astype(np.uint8)

# Method 2: Hounsfield Unit Segmentation with Color Mapping
def apply_hu_segmentation(ct_image):
    bone_segmented = ct_image > 500  # Example: Bone segmentation (HU > 500)
    colored_bone = cm.jet(bone_segmented.astype(np.float32))
    return (colored_bone[:, :, :3] * 255).astype(np.uint8)

# Method 3: LUT (Lookup Table) Color Mapping (Use predefined LUT)
def apply_lut_color_map(ct_image):
    colored_image = cm.plasma((ct_image - np.min(ct_image)) / (np.max(ct_image) - np.min(ct_image)))
    return (colored_image[:, :, :3] * 255).astype(np.uint8)

# Method 4: Threshold-Based Segmentation with Alpha Blending
def apply_alpha_blending(ct_image):
    thresholded_image = ct_image > 100  # Example threshold
    alpha = 0.5
    overlay = apply_color_map(ct_image)
    final_image = (1 - alpha) * ct_image + alpha * overlay
    return final_image.astype(np.uint8)

# Method 5: 3D Rendering (Simply slice rendering here for 2D images)
def apply_3d_rendering(ct_image):
    # For 2D images, simply simulate 3D slicing
    return apply_color_map(ct_image)

# Method 6: Gradient Color Mapping for Edge Detection
def apply_edge_detection(ct_image):
    edges = cv2.Sobel(ct_image, cv2.CV_64F, 1, 0, ksize=3)
    edges_colored = cm.jet((edges - np.min(edges)) / (np.max(edges) - np.min(edges)))
    return (edges_colored[:, :, :3] * 255).astype(np.uint8)

# Method 7: Texture Mapping (Gray Level Co-occurrence Matrix)
def apply_texture_mapping(ct_image):
    glcm = greycomatrix(ct_image, [1], [0], symmetric=True, normed=True)
    contrast = greycoprops(glcm, 'contrast')
    contrast_colormap = cm.jet((contrast - np.min(contrast)) / (np.max(contrast) - np.min(contrast)))
    return (contrast_colormap[:, :, :3] * 255).astype(np.uint8)

# Method 8: Heatmap Visualization for Anomaly Detection
def apply_heatmap(ct_image):
    heatmap_image = cm.hot((ct_image - np.min(ct_image)) / (np.max(ct_image) - np.min(ct_image)))
    return (heatmap_image[:, :, :3] * 255).astype(np.uint8)

# Method 9: Color-Mapped 3D Volume Rendering
def apply_3d_volume_rendering(ct_image):
    # Simulate 3D rendering by applying color map to a few slices (2D only for simplicity)
    return apply_color_map(ct_image)

# Method 10: Interactive Visualization with Segmented Regions
def apply_interactive_visualization(ct_image):
    # Example: Segment the image and apply color to the segmented region
    segmented_image = ct_image > 100  # Example: Threshold segmentation
    segmented_colored = cm.plasma(segmented_image.astype(np.float32))
    return (segmented_colored[:, :, :3] * 255).astype(np.uint8)

# Process each image file
for image_file in image_files:
    try:
        image_path = os.path.join(input_directory, image_file)
        
        # Load the image using PIL
        image = Image.open(image_path)
        
        # Convert image to grayscale
        ct_image = np.array(image.convert('L'))  # Convert to grayscale
        
        # Apply all methods and save the output images
        methods = [
            ("Basic Color Map", apply_color_map(ct_image, cmap='jet')),
            ("HU Segmentation", apply_hu_segmentation(ct_image)),
            ("LUT Color Map", apply_lut_color_map(ct_image)),
            ("Alpha Blending", apply_alpha_blending(ct_image)),
            ("3D Rendering", apply_3d_rendering(ct_image)),
            ("Edge Detection", apply_edge_detection(ct_image)),
            ("Texture Mapping", apply_texture_mapping(ct_image)),
            ("Heatmap Visualization", apply_heatmap(ct_image)),
            ("3D Volume Rendering", apply_3d_volume_rendering(ct_image)),
            ("Interactive Segmentation", apply_interactive_visualization(ct_image))
        ]
        
        # Save the images
        for method_name, result_image in methods:
            output_image_path = os.path.join(output_directory, f"{image_file.replace('.jpg', '')}_{method_name.replace(' ', '_')}.png")
            Image.fromarray(result_image).save(output_image_path)
            print(f"Processed and saved {image_file} with method '{method_name}' as {output_image_path}")
    
    except Exception as e:
        print(f"Error processing {image_file}: {e}")

print("Processing complete.")


ImportError: cannot import name 'greycomatrix' from 'skimage.feature' (D:\anapy\Lib\site-packages\skimage\feature\__init__.py)

In [8]:
!pip install --upgrade scikit-image


   ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/12.8 MB 653.6 kB/s eta 0:00:20
   ---------------------------------------- 0.1/12.8 MB 1.0 MB/s eta 0:00:13
    --------------------------------------- 0.2/12.8 MB 1.6 MB/s eta 0:00:08
   - -------------------------------------- 0.4/12.8 MB 2.4 MB/s eta 0:00:06
   -- ------------------------------------- 0.8/12.8 MB 3.2 MB/s eta 0:00:04
   --- ------------------------------------ 1.3/12.8 MB 4.4 MB/s eta 0:00:03
   ----- ---------------------------------- 1.7/12.8 MB 5.5 MB/s eta 0:00:03
   ----- ---------------------------------- 1.8/12.8 MB 5.7 MB/s eta 0:00:02
   ----- ---------------------------------- 1.8/12.8 MB 5.7 MB/s eta 0:00:02
   ---------- ----------------------------- 3.3/12.8 MB 7.2 MB/s eta 0:00:02
   ----------- ---------------------------- 3.6/12.8 MB 7.7 MB/s eta 0:00:02
   --------

  You can safely remove it manually.


In [9]:
import os
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
from PIL import Image
import cv2
import SimpleITK as sitk

# Define input and output directories
input_directory = r'F:\MLpractrice\ct\data'
output_directory = r'F:\MLpractrice\ct\output'

# Create the output directory if it doesn't exist
if not os.path.exists(output_directory):
    os.makedirs(output_directory)

# List all image files (jpg and png) in the directory
image_files = [f for f in os.listdir(input_directory) if f.endswith('.jpg') or f.endswith('.png')]

# Debugging: Check if image files are found
if not image_files:
    print("No image files found in the specified directory.")
else:
    print(f"Found {len(image_files)} image files.")

# Method 1: Basic Color Map Application
def apply_color_map(ct_image, cmap='jet'):
    ct_image_normalized = (ct_image - np.min(ct_image)) / (np.max(ct_image) - np.min(ct_image))
    colored_image = cm.get_cmap(cmap)(ct_image_normalized)
    return (colored_image[:, :, :3] * 255).astype(np.uint8)

# Method 2: Hounsfield Unit Segmentation with Color Mapping
def apply_hu_segmentation(ct_image):
    bone_segmented = ct_image > 500  # Example: Bone segmentation (HU > 500)
    colored_bone = cm.jet(bone_segmented.astype(np.float32))
    return (colored_bone[:, :, :3] * 255).astype(np.uint8)

# Method 3: LUT (Lookup Table) Color Mapping (Use predefined LUT)
def apply_lut_color_map(ct_image):
    colored_image = cm.plasma((ct_image - np.min(ct_image)) / (np.max(ct_image) - np.min(ct_image)))
    return (colored_image[:, :, :3] * 255).astype(np.uint8)

# Method 4: Threshold-Based Segmentation with Alpha Blending
def apply_alpha_blending(ct_image):
    thresholded_image = ct_image > 100  # Example threshold
    alpha = 0.5
    overlay = apply_color_map(ct_image)
    final_image = (1 - alpha) * ct_image + alpha * overlay
    return final_image.astype(np.uint8)

# Method 5: 3D Rendering (Simply slice rendering here for 2D images)
def apply_3d_rendering(ct_image):
    # For 2D images, simply simulate 3D slicing
    return apply_color_map(ct_image)

# Method 6: Gradient Color Mapping for Edge Detection
def apply_edge_detection(ct_image):
    edges = cv2.Sobel(ct_image, cv2.CV_64F, 1, 0, ksize=3)
    edges_colored = cm.jet((edges - np.min(edges)) / (np.max(edges) - np.min(edges)))
    return (edges_colored[:, :, :3] * 255).astype(np.uint8)

# Method 7: Heatmap Visualization for Anomaly Detection
def apply_heatmap(ct_image):
    heatmap_image = cm.hot((ct_image - np.min(ct_image)) / (np.max(ct_image) - np.min(ct_image)))
    return (heatmap_image[:, :, :3] * 255).astype(np.uint8)

# Method 8: Color-Mapped 3D Volume Rendering
def apply_3d_volume_rendering(ct_image):
    # Simulate 3D rendering by applying color map to a few slices (2D only for simplicity)
    return apply_color_map(ct_image)

# Method 9: Interactive Visualization with Segmented Regions
def apply_interactive_visualization(ct_image):
    # Example: Segment the image and apply color to the segmented region
    segmented_image = ct_image > 100  # Example: Threshold segmentation
    segmented_colored = cm.plasma(segmented_image.astype(np.float32))
    return (segmented_colored[:, :, :3] * 255).astype(np.uint8)

# Process each image file
for image_file in image_files:
    try:
        image_path = os.path.join(input_directory, image_file)
        
        # Load the image using PIL
        image = Image.open(image_path)
        
        # Convert image to grayscale
        ct_image = np.array(image.convert('L'))  # Convert to grayscale
        
        # Apply all methods and save the output images
        methods = [
            ("Basic Color Map", apply_color_map(ct_image, cmap='jet')),
            ("HU Segmentation", apply_hu_segmentation(ct_image)),
            ("LUT Color Map", apply_lut_color_map(ct_image)),
            ("Alpha Blending", apply_alpha_blending(ct_image)),
            ("3D Rendering", apply_3d_rendering(ct_image)),
            ("Edge Detection", apply_edge_detection(ct_image)),
            ("Heatmap Visualization", apply_heatmap(ct_image)),
            ("3D Volume Rendering", apply_3d_volume_rendering(ct_image)),
            ("Interactive Segmentation", apply_interactive_visualization(ct_image))
        ]
        
        # Save the images
        for method_name, result_image in methods:
            output_image_path = os.path.join(output_directory, f"{image_file.replace('.jpg', '')}_{method_name.replace(' ', '_')}.png")
            Image.fromarray(result_image).save(output_image_path)
            print(f"Processed and saved {image_file} with method '{method_name}' as {output_image_path}")
    
    except Exception as e:
        print(f"Error processing {image_file}: {e}")

print("Processing complete.")


ModuleNotFoundError: No module named 'SimpleITK'

In [10]:
import SimpleITK as sitk


ModuleNotFoundError: No module named 'SimpleITK'

In [11]:
import os
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
from PIL import Image
import cv2

# Define input and output directories
input_directory = r'F:\MLpractrice\ct\data'
output_directory = r'F:\MLpractrice\ct\output'

# Create the output directory if it doesn't exist
if not os.path.exists(output_directory):
    os.makedirs(output_directory)

# List all image files (jpg and png) in the directory
image_files = [f for f in os.listdir(input_directory) if f.endswith('.jpg') or f.endswith('.png')]

# Debugging: Check if image files are found
if not image_files:
    print("No image files found in the specified directory.")
else:
    print(f"Found {len(image_files)} image files.")

# Method 1: Basic Color Map Application
def apply_color_map(ct_image, cmap='jet'):
    ct_image_normalized = (ct_image - np.min(ct_image)) / (np.max(ct_image) - np.min(ct_image))
    colored_image = cm.get_cmap(cmap)(ct_image_normalized)
    return (colored_image[:, :, :3] * 255).astype(np.uint8)

# Method 2: Hounsfield Unit Segmentation with Color Mapping
def apply_hu_segmentation(ct_image):
    bone_segmented = ct_image > 500  # Example: Bone segmentation (HU > 500)
    colored_bone = cm.jet(bone_segmented.astype(np.float32))
    return (colored_bone[:, :, :3] * 255).astype(np.uint8)

# Method 3: LUT (Lookup Table) Color Mapping (Use predefined LUT)
def apply_lut_color_map(ct_image):
    colored_image = cm.plasma((ct_image - np.min(ct_image)) / (np.max(ct_image) - np.min(ct_image)))
    return (colored_image[:, :, :3] * 255).astype(np.uint8)

# Method 4: Threshold-Based Segmentation with Alpha Blending
def apply_alpha_blending(ct_image):
    thresholded_image = ct_image > 100  # Example threshold
    alpha = 0.5
    overlay = apply_color_map(ct_image)
    final_image = (1 - alpha) * ct_image + alpha * overlay
    return final_image.astype(np.uint8)

# Method 5: 3D Rendering (Simply slice rendering here for 2D images)
def apply_3d_rendering(ct_image):
    # For 2D images, simply simulate 3D slicing
    return apply_color_map(ct_image)

# Method 6: Gradient Color Mapping for Edge Detection
def apply_edge_detection(ct_image):
    edges = cv2.Sobel(ct_image, cv2.CV_64F, 1, 0, ksize=3)
    edges_colored = cm.jet((edges - np.min(edges)) / (np.max(edges) - np.min(edges)))
    return (edges_colored[:, :, :3] * 255).astype(np.uint8)

# Method 7: Heatmap Visualization for Anomaly Detection
def apply_heatmap(ct_image):
    heatmap_image = cm.hot((ct_image - np.min(ct_image)) / (np.max(ct_image) - np.min(ct_image)))
    return (heatmap_image[:, :, :3] * 255).astype(np.uint8)

# Method 8: Color-Mapped 3D Volume Rendering
def apply_3d_volume_rendering(ct_image):
    # Simulate 3D rendering by applying color map to a few slices (2D only for simplicity)
    return apply_color_map(ct_image)

# Method 9: Interactive Visualization with Segmented Regions
def apply_interactive_visualization(ct_image):
    # Example: Segment the image and apply color to the segmented region
    segmented_image = ct_image > 100  # Example: Threshold segmentation
    segmented_colored = cm.plasma(segmented_image.astype(np.float32))
    return (segmented_colored[:, :, :3] * 255).astype(np.uint8)

# Process each image file
for image_file in image_files:
    try:
        image_path = os.path.join(input_directory, image_file)
        
        # Load the image using PIL
        image = Image.open(image_path)
        
        # Convert image to grayscale
        ct_image = np.array(image.convert('L'))  # Convert to grayscale
        
        # Apply all methods and save the output images
        methods = [
            ("Basic Color Map", apply_color_map(ct_image, cmap='jet')),
            ("HU Segmentation", apply_hu_segmentation(ct_image)),
            ("LUT Color Map", apply_lut_color_map(ct_image)),
            ("Alpha Blending", apply_alpha_blending(ct_image)),
            ("3D Rendering", apply_3d_rendering(ct_image)),
            ("Edge Detection", apply_edge_detection(ct_image)),
            ("Heatmap Visualization", apply_heatmap(ct_image)),
            ("3D Volume Rendering", apply_3d_volume_rendering(ct_image)),
            ("Interactive Segmentation", apply_interactive_visualization(ct_image))
        ]
        
        # Save the images
        for method_name, result_image in methods:
            output_image_path = os.path.join(output_directory, f"{image_file.replace('.jpg', '')}_{method_name.replace(' ', '_')}.png")
            Image.fromarray(result_image).save(output_image_path)
            print(f"Processed and saved {image_file} with method '{method_name}' as {output_image_path}")
    
    except Exception as e:
        print(f"Error processing {image_file}: {e}")

print("Processing complete.")


Found 2 image files.
Error processing 2.png: operands could not be broadcast together with shapes (626,630) (626,630,3) 


C:\Users\AIC\AppData\Local\Temp\ipykernel_14036\2100446393.py:28: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  colored_image = cm.get_cmap(cmap)(ct_image_normalized)


Error processing istockphoto-502727050-612x612.jpg: operands could not be broadcast together with shapes (367,612) (367,612,3) 
Processing complete.


In [12]:
import os
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
from PIL import Image
import cv2

# Define input and output directories
input_directory = r'F:\MLpractrice\ct\data'
output_directory = r'F:\MLpractrice\ct\output'

# Create the output directory if it doesn't exist
if not os.path.exists(output_directory):
    os.makedirs(output_directory)

# List all image files (jpg and png) in the directory
image_files = [f for f in os.listdir(input_directory) if f.endswith('.jpg') or f.endswith('.png')]

# Debugging: Check if image files are found
if not image_files:
    print("No image files found in the specified directory.")
else:
    print(f"Found {len(image_files)} image files.")

# Method 1: Basic Color Map Application
def apply_color_map(ct_image, cmap='jet'):
    # Normalize the grayscale image to the range [0, 1]
    ct_image_normalized = (ct_image - np.min(ct_image)) / (np.max(ct_image) - np.min(ct_image))
    
    # Apply color map to the normalized image (returning a 3D RGB image)
    colored_image = cm.get_cmap(cmap)(ct_image_normalized)
    
    # Ensure the image is in 8-bit format and has 3 channels
    return (colored_image[:, :, :3] * 255).astype(np.uint8)

# Method 2: Hounsfield Unit Segmentation with Color Mapping
def apply_hu_segmentation(ct_image):
    bone_segmented = ct_image > 500  # Example: Bone segmentation (HU > 500)
    colored_bone = cm.jet(bone_segmented.astype(np.float32))
    return (colored_bone[:, :, :3] * 255).astype(np.uint8)

# Method 3: LUT (Lookup Table) Color Mapping (Use predefined LUT)
def apply_lut_color_map(ct_image):
    colored_image = cm.plasma((ct_image - np.min(ct_image)) / (np.max(ct_image) - np.min(ct_image)))
    return (colored_image[:, :, :3] * 255).astype(np.uint8)

# Method 4: Threshold-Based Segmentation with Alpha Blending
def apply_alpha_blending(ct_image):
    thresholded_image = ct_image > 100  # Example threshold
    
    # Apply color map to the thresholded image
    colored_image = apply_color_map(ct_image)
    
    # Ensure both images have the same shape
    thresholded_image_rgb = np.stack([thresholded_image] * 3, axis=-1)  # Convert to RGB
    
    # Perform alpha blending
    alpha = 0.5
    final_image = alpha * colored_image + (1 - alpha) * thresholded_image_rgb * 255
    
    # Ensure the result is uint8
    return final_image.astype(np.uint8)

# Method 5: 3D Rendering (Simply slice rendering here for 2D images)
def apply_3d_rendering(ct_image):
    return apply_color_map(ct_image)

# Method 6: Gradient Color Mapping for Edge Detection
def apply_edge_detection(ct_image):
    edges = cv2.Sobel(ct_image, cv2.CV_64F, 1, 0, ksize=3)
    edges_colored = cm.jet((edges - np.min(edges)) / (np.max(edges) - np.min(edges)))
    return (edges_colored[:, :, :3] * 255).astype(np.uint8)

# Method 7: Heatmap Visualization for Anomaly Detection
def apply_heatmap(ct_image):
    heatmap_image = cm.hot((ct_image - np.min(ct_image)) / (np.max(ct_image) - np.min(ct_image)))
    return (heatmap_image[:, :, :3] * 255).astype(np.uint8)

# Method 8: Color-Mapped 3D Volume Rendering
def apply_3d_volume_rendering(ct_image):
    return apply_color_map(ct_image)

# Method 9: Interactive Visualization with Segmented Regions
def apply_interactive_visualization(ct_image):
    segmented_image = ct_image > 100  # Example: Threshold segmentation
    segmented_colored = cm.plasma(segmented_image.astype(np.float32))
    return (segmented_colored[:, :, :3] * 255).astype(np.uint8)

# Process each image file
for image_file in image_files:
    try:
        image_path = os.path.join(input_directory, image_file)
        
        # Load the image using PIL
        image = Image.open(image_path)
        
        # Convert image to grayscale
        ct_image = np.array(image.convert('L'))  # Convert to grayscale
        
        # Apply all methods and save the output images
        methods = [
            ("Basic Color Map", apply_color_map(ct_image, cmap='jet')),
            ("HU Segmentation", apply_hu_segmentation(ct_image)),
            ("LUT Color Map", apply_lut_color_map(ct_image)),
            ("Alpha Blending", apply_alpha_blending(ct_image)),
            ("3D Rendering", apply_3d_rendering(ct_image)),
            ("Edge Detection", apply_edge_detection(ct_image)),
            ("Heatmap Visualization", apply_heatmap(ct_image)),
            ("3D Volume Rendering", apply_3d_volume_rendering(ct_image)),
            ("Interactive Segmentation", apply_interactive_visualization(ct_image))
        ]
        
        # Save the images
        for method_name, result_image in methods:
            output_image_path = os.path.join(output_directory, f"{image_file.replace('.jpg', '')}_{method_name.replace(' ', '_')}.png")
            Image.fromarray(result_image).save(output_image_path)
            print(f"Processed and saved {image_file} with method '{method_name}' as {output_image_path}")
    
    except Exception as e:
        print(f"Error processing {image_file}: {e}")

print("Processing complete.")


Found 2 image files.


C:\Users\AIC\AppData\Local\Temp\ipykernel_14036\1753662389.py:31: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  colored_image = cm.get_cmap(cmap)(ct_image_normalized)


Processed and saved 2.png with method 'Basic Color Map' as F:\MLpractrice\ct\output\2.png_Basic_Color_Map.png
Processed and saved 2.png with method 'HU Segmentation' as F:\MLpractrice\ct\output\2.png_HU_Segmentation.png
Processed and saved 2.png with method 'LUT Color Map' as F:\MLpractrice\ct\output\2.png_LUT_Color_Map.png
Processed and saved 2.png with method 'Alpha Blending' as F:\MLpractrice\ct\output\2.png_Alpha_Blending.png
Processed and saved 2.png with method '3D Rendering' as F:\MLpractrice\ct\output\2.png_3D_Rendering.png
Processed and saved 2.png with method 'Edge Detection' as F:\MLpractrice\ct\output\2.png_Edge_Detection.png
Processed and saved 2.png with method 'Heatmap Visualization' as F:\MLpractrice\ct\output\2.png_Heatmap_Visualization.png
Processed and saved 2.png with method '3D Volume Rendering' as F:\MLpractrice\ct\output\2.png_3D_Volume_Rendering.png
Processed and saved 2.png with method 'Interactive Segmentation' as F:\MLpractrice\ct\output\2.png_Interactive_Seg

In [13]:
import os
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
from PIL import Image
import cv2

# Define input and output directories
input_directory = r'F:\MLpractrice\ct\data'
output_directory = r'F:\MLpractrice\ct\output'

# Create the output directory if it doesn't exist
if not os.path.exists(output_directory):
    os.makedirs(output_directory)

# List all image files (jpg and png) in the directory
image_files = [f for f in os.listdir(input_directory) if f.endswith('.jpg') or f.endswith('.png')]

# Debugging: Check if image files are found
if not image_files:
    print("No image files found in the specified directory.")
else:
    print(f"Found {len(image_files)} image files.")

# Method 1: Basic Color Map Application
def apply_color_map(ct_image, cmap='jet'):
    # Normalize the grayscale image to the range [0, 1]
    ct_image_normalized = (ct_image - np.min(ct_image)) / (np.max(ct_image) - np.min(ct_image))
    
    # Apply color map to the normalized image (returning a 3D RGB image)
    colored_image = cm.get_cmap(cmap)(ct_image_normalized)
    
    # Ensure the image is in 8-bit format and has 3 channels
    return (colored_image[:, :, :3] * 255).astype(np.uint8)

# Method 2: LUT (Lookup Table) Color Mapping (Use predefined LUT)
def apply_lut_color_map(ct_image):
    colored_image = cm.plasma((ct_image - np.min(ct_image)) / (np.max(ct_image) - np.min(ct_image)))
    return (colored_image[:, :, :3] * 255).astype(np.uint8)

# Method 3: Threshold-Based Segmentation with Alpha Blending
def apply_alpha_blending(ct_image):
    thresholded_image = ct_image > 100  # Example threshold
    
    # Apply color map to the thresholded image
    colored_image = apply_color_map(ct_image)
    
    # Ensure both images have the same shape
    thresholded_image_rgb = np.stack([thresholded_image] * 3, axis=-1)  # Convert to RGB
    
    # Perform alpha blending
    alpha = 0.5
    final_image = alpha * colored_image + (1 - alpha) * thresholded_image_rgb * 255
    
    # Ensure the result is uint8
    return final_image.astype(np.uint8)

# Method 4: 3D Rendering (Simply slice rendering here for 2D images)
def apply_3d_rendering(ct_image):
    return apply_color_map(ct_image)

# Method 5: Gradient Color Mapping for Edge Detection
def apply_edge_detection(ct_image):
    edges = cv2.Sobel(ct_image, cv2.CV_64F, 1, 0, ksize=3)
    edges_colored = cm.jet((edges - np.min(edges)) / (np.max(edges) - np.min(edges)))
    return (edges_colored[:, :, :3] * 255).astype(np.uint8)

# Method 6: Heatmap Visualization for Anomaly Detection
def apply_heatmap(ct_image):
    heatmap_image = cm.hot((ct_image - np.min(ct_image)) / (np.max(ct_image) - np.min(ct_image)))
    return (heatmap_image[:, :, :3] * 255).astype(np.uint8)

# Method 7: Color-Mapped 3D Volume Rendering
def apply_3d_volume_rendering(ct_image):
    return apply_color_map(ct_image)

# Method 8: Interactive Visualization with Segmented Regions
def apply_interactive_visualization(ct_image):
    segmented_image = ct_image > 100  # Example: Threshold segmentation
    segmented_colored = cm.plasma(segmented_image.astype(np.float32))
    return (segmented_colored[:, :, :3] * 255).astype(np.uint8)

# Process each image file
for image_file in image_files:
    try:
        image_path = os.path.join(input_directory, image_file)
        
        # Load the image using PIL
        image = Image.open(image_path)
        
        # Convert image to grayscale
        ct_image = np.array(image.convert('L'))  # Convert to grayscale
        
        # Apply all methods and save the output images
        methods = [
            ("Basic Color Map", apply_color_map(ct_image, cmap='jet')),
            ("LUT Color Map", apply_lut_color_map(ct_image)),
            ("Alpha Blending", apply_alpha_blending(ct_image)),
            ("3D Rendering", apply_3d_rendering(ct_image)),
            ("Edge Detection", apply_edge_detection(ct_image)),
            ("Heatmap Visualization", apply_heatmap(ct_image)),
            ("3D Volume Rendering", apply_3d_volume_rendering(ct_image)),
            ("Interactive Segmentation", apply_interactive_visualization(ct_image))
        ]
        
        # Save the images
        for method_name, result_image in methods:
            output_image_path = os.path.join(output_directory, f"{image_file.replace('.jpg', '')}_{method_name.replace(' ', '_')}.png")
            Image.fromarray(result_image).save(output_image_path)
            print(f"Processed and saved {image_file} with method '{method_name}' as {output_image_path}")
    
    except Exception as e:
        print(f"Error processing {image_file}: {e}")

print("Processing complete.")


Found 2 image files.


C:\Users\AIC\AppData\Local\Temp\ipykernel_14036\2987998485.py:31: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  colored_image = cm.get_cmap(cmap)(ct_image_normalized)


Processed and saved 2.png with method 'Basic Color Map' as F:\MLpractrice\ct\output\2.png_Basic_Color_Map.png
Processed and saved 2.png with method 'LUT Color Map' as F:\MLpractrice\ct\output\2.png_LUT_Color_Map.png
Processed and saved 2.png with method 'Alpha Blending' as F:\MLpractrice\ct\output\2.png_Alpha_Blending.png
Processed and saved 2.png with method '3D Rendering' as F:\MLpractrice\ct\output\2.png_3D_Rendering.png
Processed and saved 2.png with method 'Edge Detection' as F:\MLpractrice\ct\output\2.png_Edge_Detection.png
Processed and saved 2.png with method 'Heatmap Visualization' as F:\MLpractrice\ct\output\2.png_Heatmap_Visualization.png
Processed and saved 2.png with method '3D Volume Rendering' as F:\MLpractrice\ct\output\2.png_3D_Volume_Rendering.png
Processed and saved 2.png with method 'Interactive Segmentation' as F:\MLpractrice\ct\output\2.png_Interactive_Segmentation.png
Processed and saved istockphoto-502727050-612x612.jpg with method 'Basic Color Map' as F:\MLprac

In [ ]:
Basic Color Map Application
LUT Color Map
Alpha Blending
3D Rendering
Edge Detection
Heatmap Visualization
3D Volume Rendering
Interactive Segmentation

In [14]:
import os
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
from PIL import Image
import cv2

# Define input and output directories
input_directory = r'F:\MLpractrice\ct\data'
output_directory = r'F:\MLpractrice\ct\output'

# Create the output directory if it doesn't exist
if not os.path.exists(output_directory):
    os.makedirs(output_directory)

# List all image files (jpg and png) in the directory
image_files = [f for f in os.listdir(input_directory) if f.endswith('.jpg') or f.endswith('.png')]

# Debugging: Check if image files are found
if not image_files:
    print("No image files found in the specified directory.")
else:
    print(f"Found {len(image_files)} image files.")

# Method 1: Basic Color Map Application
def apply_color_map(ct_image, cmap='jet'):
    # Normalize the grayscale image to the range [0, 1]
    ct_image_normalized = (ct_image - np.min(ct_image)) / (np.max(ct_image) - np.min(ct_image))
    
    # Apply color map to the normalized image (returning a 3D RGB image)
    colored_image = cm.get_cmap(cmap)(ct_image_normalized)
    
    # Ensure the image is in 8-bit format and has 3 channels
    return (colored_image[:, :, :3] * 255).astype(np.uint8)

# Method 2: Adaptive Histogram Equalization with Color Map
def apply_adaptive_hist_eq(ct_image):
    # Apply CLAHE (Contrast Limited Adaptive Histogram Equalization)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    enhanced_image = clahe.apply(ct_image)
    
    # Apply a color map to the enhanced image
    return apply_color_map(enhanced_image)

# Method 3: Contrast Stretching with Color Map
def apply_contrast_stretching(ct_image):
    min_val, max_val = np.min(ct_image), np.max(ct_image)
    stretched_image = (ct_image - min_val) / (max_val - min_val)  # Stretch to [0, 1]
    return apply_color_map(stretched_image)

# Method 4: Gaussian Blur with Color Map
def apply_gaussian_blur(ct_image):
    blurred_image = cv2.GaussianBlur(ct_image, (5, 5), 0)
    return apply_color_map(blurred_image)

# Method 5: Edge Detection with Canny and Color Map
def apply_edge_detection(ct_image):
    edges = cv2.Canny(ct_image, 100, 200)  # Canny edge detection
    return apply_color_map(edges)

# Method 6: Random Color Palettes (Color Overlay)
def apply_random_color_palette(ct_image):
    random_colored = np.random.rand(*ct_image.shape, 3)  # Generate random colors
    colored_image = ct_image[:, :, None] * random_colored  # Overlay on the CT scan intensities
    return (colored_image * 255).astype(np.uint8)

# Method 7: Non-Linear Color Stretching (Gamma Correction)
def apply_gamma_correction(ct_image, gamma=1.2):
    # Apply Gamma correction to stretch intensities
    gamma_corrected = np.power(ct_image / float(np.max(ct_image)), gamma) * 255
    return apply_color_map(gamma_corrected)

# Process each image file
for image_file in image_files:
    try:
        image_path = os.path.join(input_directory, image_file)
        
        # Load the image using PIL
        image = Image.open(image_path)
        
        # Convert image to grayscale
        ct_image = np.array(image.convert('L'))  # Convert to grayscale
        
        # Apply all methods and save the output images
        methods = [
            ("Basic Color Map", apply_color_map(ct_image, cmap='jet')),
            ("Adaptive Histogram Equalization", apply_adaptive_hist_eq(ct_image)),
            ("Contrast Stretching", apply_contrast_stretching(ct_image)),
            ("Gaussian Blur", apply_gaussian_blur(ct_image)),
            ("Edge Detection", apply_edge_detection(ct_image)),
            ("Random Color Palette", apply_random_color_palette(ct_image)),
            ("Gamma Correction", apply_gamma_correction(ct_image, gamma=1.5))
        ]
        
        # Save the images
        for method_name, result_image in methods:
            output_image_path = os.path.join(output_directory, f"{image_file.replace('.jpg', '')}_{method_name.replace(' ', '_')}.png")
            Image.fromarray(result_image).save(output_image_path)
            print(f"Processed and saved {image_file} with method '{method_name}' as {output_image_path}")
    
    except Exception as e:
        print(f"Error processing {image_file}: {e}")

print("Processing complete.")


Found 2 image files.


C:\Users\AIC\AppData\Local\Temp\ipykernel_14036\3466112618.py:31: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  colored_image = cm.get_cmap(cmap)(ct_image_normalized)


Processed and saved 2.png with method 'Basic Color Map' as F:\MLpractrice\ct\output\2.png_Basic_Color_Map.png
Processed and saved 2.png with method 'Adaptive Histogram Equalization' as F:\MLpractrice\ct\output\2.png_Adaptive_Histogram_Equalization.png
Processed and saved 2.png with method 'Contrast Stretching' as F:\MLpractrice\ct\output\2.png_Contrast_Stretching.png
Processed and saved 2.png with method 'Gaussian Blur' as F:\MLpractrice\ct\output\2.png_Gaussian_Blur.png
Processed and saved 2.png with method 'Edge Detection' as F:\MLpractrice\ct\output\2.png_Edge_Detection.png
Processed and saved 2.png with method 'Random Color Palette' as F:\MLpractrice\ct\output\2.png_Random_Color_Palette.png
Processed and saved 2.png with method 'Gamma Correction' as F:\MLpractrice\ct\output\2.png_Gamma_Correction.png
Processed and saved istockphoto-502727050-612x612.jpg with method 'Basic Color Map' as F:\MLpractrice\ct\output\istockphoto-502727050-612x612_Basic_Color_Map.png
Processed and saved is

In [15]:
import os
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
from PIL import Image
import cv2

# Define input and output directories
input_directory = r'F:\MLpractrice\ct\data'
output_directory = r'F:\MLpractrice\ct\output'

# Create the output directory if it doesn't exist
if not os.path.exists(output_directory):
    os.makedirs(output_directory)

# List all image files (jpg and png) in the directory
image_files = [f for f in os.listdir(input_directory) if f.endswith('.jpg') or f.endswith('.png')]

# Debugging: Check if image files are found
if not image_files:
    print("No image files found in the specified directory.")
else:
    print(f"Found {len(image_files)} image files.")

# Method 1: Basic Color Map Application
def apply_color_map(ct_image, cmap='jet'):
    # Normalize the grayscale image to the range [0, 1]
    ct_image_normalized = (ct_image - np.min(ct_image)) / (np.max(ct_image) - np.min(ct_image))
    
    # Apply color map to the normalized image (returning a 3D RGB image)
    colored_image = cm.get_cmap(cmap)(ct_image_normalized)
    
    # Ensure the image is in 8-bit format and has 3 channels
    return (colored_image[:, :, :3] * 255).astype(np.uint8)

# Method 2: Adaptive Histogram Equalization with Color Map
def apply_adaptive_hist_eq(ct_image):
    # Apply CLAHE (Contrast Limited Adaptive Histogram Equalization)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    enhanced_image = clahe.apply(ct_image)
    
    # Apply a color map to the enhanced image
    return apply_color_map(enhanced_image)

# Method 3: Contrast Stretching with Color Map
def apply_contrast_stretching(ct_image):
    min_val, max_val = np.min(ct_image), np.max(ct_image)
    stretched_image = (ct_image - min_val) / (max_val - min_val)  # Stretch to [0, 1]
    return apply_color_map(stretched_image)

# Method 4: Gaussian Blur with Color Map
def apply_gaussian_blur(ct_image):
    blurred_image = cv2.GaussianBlur(ct_image, (5, 5), 0)
    return apply_color_map(blurred_image)

# Method 5: Edge Detection with Canny and Color Map
def apply_edge_detection(ct_image):
    edges = cv2.Canny(ct_image, 100, 200)  # Canny edge detection
    return apply_color_map(edges)

# Method 6: Random Color Palettes (Color Overlay)
def apply_random_color_palette(ct_image):
    random_colored = np.random.rand(*ct_image.shape, 3)  # Generate random colors
    colored_image = ct_image[:, :, None] * random_colored  # Overlay on the CT scan intensities
    return (colored_image * 255).astype(np.uint8)

# Method 7: Non-Linear Color Stretching (Gamma Correction)
def apply_gamma_correction(ct_image, gamma=1.2):
    # Apply Gamma correction to stretch intensities
    gamma_corrected = np.power(ct_image / float(np.max(ct_image)), gamma) * 255
    return apply_color_map(gamma_corrected)

# Method 8: LUT (Lookup Table) Color Mapping (Use predefined LUT)
def apply_lut_color_map(ct_image):
    colored_image = cm.plasma((ct_image - np.min(ct_image)) / (np.max(ct_image) - np.min(ct_image)))
    return (colored_image[:, :, :3] * 255).astype(np.uint8)

# Method 9: Alpha Blending (Threshold-based)
def apply_alpha_blending(ct_image):
    thresholded_image = ct_image > 100  # Example threshold
    
    # Apply color map to the thresholded image
    colored_image = apply_color_map(ct_image)
    
    # Ensure both images have the same shape
    thresholded_image_rgb = np.stack([thresholded_image] * 3, axis=-1)  # Convert to RGB
    
    # Perform alpha blending
    alpha = 0.5
    final_image = alpha * colored_image + (1 - alpha) * thresholded_image_rgb * 255
    
    # Ensure the result is uint8
    return final_image.astype(np.uint8)

# Method 10: 3D Rendering (Simply slice rendering here for 2D images)
def apply_3d_rendering(ct_image):
    return apply_color_map(ct_image)

# Method 11: Heatmap Visualization for Anomaly Detection
def apply_heatmap(ct_image):
    heatmap_image = cm.hot((ct_image - np.min(ct_image)) / (np.max(ct_image) - np.min(ct_image)))
    return (heatmap_image[:, :, :3] * 255).astype(np.uint8)

# Method 12: 3D Volume Rendering (Placeholder for 3D visualization)
def apply_3d_volume_rendering(ct_image):
    return apply_color_map(ct_image)

# Method 13: Interactive Visualization with Segmented Regions
def apply_interactive_visualization(ct_image):
    segmented_image = ct_image > 100  # Example: Threshold segmentation
    segmented_colored = cm.plasma(segmented_image.astype(np.float32))
    return (segmented_colored[:, :, :3] * 255).astype(np.uint8)

# Process each image file
for image_file in image_files:
    try:
        image_path = os.path.join(input_directory, image_file)
        
        # Load the image using PIL
        image = Image.open(image_path)
        
        # Convert image to grayscale
        ct_image = np.array(image.convert('L'))  # Convert to grayscale
        
        # Apply all methods and save the output images
        methods = [
            ("Basic Color Map", apply_color_map(ct_image, cmap='jet')),
            ("Adaptive Histogram Equalization", apply_adaptive_hist_eq(ct_image)),
            ("Contrast Stretching", apply_contrast_stretching(ct_image)),
            ("Gaussian Blur", apply_gaussian_blur(ct_image)),
            ("Edge Detection", apply_edge_detection(ct_image)),
            ("Random Color Palette", apply_random_color_palette(ct_image)),
            ("Gamma Correction", apply_gamma_correction(ct_image, gamma=1.5)),
            ("LUT Color Map", apply_lut_color_map(ct_image)),
            ("Alpha Blending", apply_alpha_blending(ct_image)),
            ("3D Rendering", apply_3d_rendering(ct_image)),
            ("Heatmap Visualization", apply_heatmap(ct_image)),
            ("3D Volume Rendering", apply_3d_volume_rendering(ct_image)),
            ("Interactive Segmentation", apply_interactive_visualization(ct_image))
        ]
        
        # Save the images
        for method_name, result_image in methods:
            output_image_path = os.path.join(output_directory, f"{image_file.replace('.jpg', '')}_{method_name.replace(' ', '_')}.png")
            Image.fromarray(result_image).save(output_image_path)
            print(f"Processed and saved {image_file} with method '{method_name}' as {output_image_path}")
    
    except Exception as e:
        print(f"Error processing {image_file}: {e}")

print("Processing complete.")


Found 2 image files.


C:\Users\AIC\AppData\Local\Temp\ipykernel_14036\1371923063.py:31: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  colored_image = cm.get_cmap(cmap)(ct_image_normalized)


Processed and saved 2.png with method 'Basic Color Map' as F:\MLpractrice\ct\output\2.png_Basic_Color_Map.png
Processed and saved 2.png with method 'Adaptive Histogram Equalization' as F:\MLpractrice\ct\output\2.png_Adaptive_Histogram_Equalization.png
Processed and saved 2.png with method 'Contrast Stretching' as F:\MLpractrice\ct\output\2.png_Contrast_Stretching.png
Processed and saved 2.png with method 'Gaussian Blur' as F:\MLpractrice\ct\output\2.png_Gaussian_Blur.png
Processed and saved 2.png with method 'Edge Detection' as F:\MLpractrice\ct\output\2.png_Edge_Detection.png
Processed and saved 2.png with method 'Random Color Palette' as F:\MLpractrice\ct\output\2.png_Random_Color_Palette.png
Processed and saved 2.png with method 'Gamma Correction' as F:\MLpractrice\ct\output\2.png_Gamma_Correction.png
Processed and saved 2.png with method 'LUT Color Map' as F:\MLpractrice\ct\output\2.png_LUT_Color_Map.png
Processed and saved 2.png with method 'Alpha Blending' as F:\MLpractrice\ct\ou

In [16]:
import os
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
from PIL import Image
import cv2

# Define input and output directories
input_directory = r'F:\MLpractrice\ct\data'
output_directory = r'F:\MLpractrice\ct\output'

# Create the output directory if it doesn't exist
if not os.path.exists(output_directory):
    os.makedirs(output_directory)

# Method 1: Basic Color Map Application
def apply_color_map(ct_image, cmap='jet'):
    ct_image_normalized = (ct_image - np.min(ct_image)) / (np.max(ct_image) - np.min(ct_image))
    colored_image = cm.get_cmap(cmap)(ct_image_normalized)
    return (colored_image[:, :, :3] * 255).astype(np.uint8)

# Method 2: Adaptive Histogram Equalization with Color Map
def apply_adaptive_hist_eq(ct_image):
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    enhanced_image = clahe.apply(ct_image)
    return apply_color_map(enhanced_image)

# Method 3: Contrast Stretching with Color Map
def apply_contrast_stretching(ct_image):
    min_val, max_val = np.min(ct_image), np.max(ct_image)
    stretched_image = (ct_image - min_val) / (max_val - min_val)
    return apply_color_map(stretched_image)

# Method 4: Gaussian Blur with Color Map
def apply_gaussian_blur(ct_image):
    blurred_image = cv2.GaussianBlur(ct_image, (5, 5), 0)
    return apply_color_map(blurred_image)

# Method 5: Edge Detection with Canny and Color Map
def apply_edge_detection(ct_image):
    edges = cv2.Canny(ct_image, 100, 200)
    return apply_color_map(edges)

# Method 6: Random Color Palettes (Color Overlay)
def apply_random_color_palette(ct_image):
    random_colored = np.random.rand(*ct_image.shape, 3)
    colored_image = ct_image[:, :, None] * random_colored
    return (colored_image * 255).astype(np.uint8)

# Method 7: Non-Linear Color Stretching (Gamma Correction)
def apply_gamma_correction(ct_image, gamma=1.2):
    gamma_corrected = np.power(ct_image / float(np.max(ct_image)), gamma) * 255
    return apply_color_map(gamma_corrected)

# Method 8: LUT (Lookup Table) Color Mapping (Use predefined LUT)
def apply_lut_color_map(ct_image):
    colored_image = cm.plasma((ct_image - np.min(ct_image)) / (np.max(ct_image) - np.min(ct_image)))
    return (colored_image[:, :, :3] * 255).astype(np.uint8)

# Method 9: Alpha Blending (Threshold-based)
def apply_alpha_blending(ct_image):
    thresholded_image = ct_image > 100
    colored_image = apply_color_map(ct_image)
    thresholded_image_rgb = np.stack([thresholded_image] * 3, axis=-1)
    alpha = 0.5
    final_image = alpha * colored_image + (1 - alpha) * thresholded_image_rgb * 255
    return final_image.astype(np.uint8)

# Method 10: 3D Rendering (Placeholder for 2D images)
def apply_3d_rendering(ct_image):
    return apply_color_map(ct_image)

# Method 11: Heatmap Visualization for Anomaly Detection
def apply_heatmap(ct_image):
    heatmap_image = cm.hot((ct_image - np.min(ct_image)) / (np.max(ct_image) - np.min(ct_image)))
    return (heatmap_image[:, :, :3] * 255).astype(np.uint8)

# Method 12: 3D Volume Rendering (Placeholder for 3D visualization)
def apply_3d_volume_rendering(ct_image):
    return apply_color_map(ct_image)

# Method 13: Interactive Visualization with Segmented Regions
def apply_interactive_visualization(ct_image):
    segmented_image = ct_image > 100
    segmented_colored = cm.plasma(segmented_image.astype(np.float32))
    return (segmented_colored[:, :, :3] * 255).astype(np.uint8)

# Traverse the input directory and process each image in the subfolders
for root, dirs, files in os.walk(input_directory):
    # For each image file in the directory
    for image_file in files:
        if image_file.endswith('.jpg') or image_file.endswith('.png'):
            try:
                # Construct the full input path for the image
                image_path = os.path.join(root, image_file)
                
                # Load the image using PIL
                image = Image.open(image_path)
                
                # Convert image to grayscale
                ct_image = np.array(image.convert('L'))  # Convert to grayscale
                
                # Create the same folder structure in the output directory
                relative_path = os.path.relpath(root, input_directory)  # Get relative folder structure
                output_folder = os.path.join(output_directory, relative_path)
                if not os.path.exists(output_folder):
                    os.makedirs(output_folder)
                
                # Apply all methods and save the output images
                methods = [
                    ("Basic Color Map", apply_color_map(ct_image, cmap='jet')),
                    ("Adaptive Histogram Equalization", apply_adaptive_hist_eq(ct_image)),
                    ("Contrast Stretching", apply_contrast_stretching(ct_image)),
                    ("Gaussian Blur", apply_gaussian_blur(ct_image)),
                    ("Edge Detection", apply_edge_detection(ct_image)),
                    ("Random Color Palette", apply_random_color_palette(ct_image)),
                    ("Gamma Correction", apply_gamma_correction(ct_image, gamma=1.5)),
                    ("LUT Color Map", apply_lut_color_map(ct_image)),
                    ("Alpha Blending", apply_alpha_blending(ct_image)),
                    ("3D Rendering", apply_3d_rendering(ct_image)),
                    ("Heatmap Visualization", apply_heatmap(ct_image)),
                    ("3D Volume Rendering", apply_3d_volume_rendering(ct_image)),
                    ("Interactive Segmentation", apply_interactive_visualization(ct_image))
                ]
                
                # Save the images as .jpg in the output folder
                for idx, (method_name, result_image) in enumerate(methods):
                    output_image_path = os.path.join(output_folder, f"image{idx+1}.jpg")
                    Image.fromarray(result_image).save(output_image_path, format='JPEG')
                    print(f"Processed and saved {image_file} with method '{method_name}' as {output_image_path}")
            
            except Exception as e:
                print(f"Error processing {image_file} in {root}: {e}")

print("Processing complete.")


C:\Users\AIC\AppData\Local\Temp\ipykernel_14036\885526207.py:19: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  colored_image = cm.get_cmap(cmap)(ct_image_normalized)


Processed and saved 2.png with method 'Basic Color Map' as F:\MLpractrice\ct\output\a\image1.jpg
Processed and saved 2.png with method 'Adaptive Histogram Equalization' as F:\MLpractrice\ct\output\a\image2.jpg
Processed and saved 2.png with method 'Contrast Stretching' as F:\MLpractrice\ct\output\a\image3.jpg
Processed and saved 2.png with method 'Gaussian Blur' as F:\MLpractrice\ct\output\a\image4.jpg
Processed and saved 2.png with method 'Edge Detection' as F:\MLpractrice\ct\output\a\image5.jpg
Processed and saved 2.png with method 'Random Color Palette' as F:\MLpractrice\ct\output\a\image6.jpg
Processed and saved 2.png with method 'Gamma Correction' as F:\MLpractrice\ct\output\a\image7.jpg
Processed and saved 2.png with method 'LUT Color Map' as F:\MLpractrice\ct\output\a\image8.jpg
Processed and saved 2.png with method 'Alpha Blending' as F:\MLpractrice\ct\output\a\image9.jpg
Processed and saved 2.png with method '3D Rendering' as F:\MLpractrice\ct\output\a\image10.jpg
Processed an